In [1]:
import sklearn
import sklearn.datasets
import sklearn.ensemble
import numpy as np
import lime
import lime.lime_tabular
from __future__ import print_function
np.random.seed(1)

In [2]:
import pandas as pd


In [3]:

vat_data=pd.read_csv('dataset_clean.xls', delimiter=";")
vat_data_inputs=vat_data[['Ship_from','Ship_to','Incoterm','Taxclass_material','Buyer','VAT_ID','Price_net','Price_gross']]
vat_data_target=vat_data['Taxcode']
vat_data_inputs.dtypes
vat_data_inputs['Incoterm']=vat_data_inputs['Incoterm'].astype('str')
vat_data_inputs['Taxclass_material']=vat_data_inputs['Taxclass_material'].astype('str')

vat_data_inputs=pd.get_dummies(vat_data_inputs)
vat_data_inputs.shape
vat_data_target=pd.get_dummies(vat_data_target)
vat_data_target.shape

c:\users\taxdev\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\taxdev\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


(10000, 2)

In [4]:
from sklearn.model_selection import GridSearchCV

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import adam
from keras.utils.np_utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier

# Keras models can be used by scikit-learn but require a build function for GridsearchCV
def create_model(layer_one_nodes=8, layer_two_nodes=4,num_h_layers=1, optimizer='sgd'):

    keras_model = Sequential()
    keras_model.add(Dense(units=layer_one_nodes, activation='relu', input_dim=vat_data_inputs.shape[1]))
    for h in range(num_h_layers):
        keras_model.add(Dense(units=layer_two_nodes, activation='relu'))

    #keras_model.add(Dense(units=layer_one_nodes, activation='relu'))

    optim=adam(lr=0.001)
    keras_model.add(Dense(units=vat_data_target.shape[1], activation='softmax'))

    keras_model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    print(keras_model.summary())
    
    return keras_model

Using TensorFlow backend.


In [ ]:
keras_model = KerasClassifier(build_fn=create_model, verbose=1, epochs=300)

keras_param_grid = {'layer_one_nodes':[20], 'layer_two_nodes': [20],'num_h_layers':[1], 'optimizer':['adam']}

#keras_param_grid = {'layer_one_nodes':[20,30,40,50], 'layer_two_nodes': [20,30,40,50],'num_h_layers':[1,2,3,4], 'optimizer':['adam']}



gs = GridSearchCV(keras_model, keras_param_grid, n_jobs=1, cv=2) 
gs.fit(vat_data_inputs.values,vat_data_target.values)

    

In [8]:
score = gs.score(vat_data_inputs.values,vat_data_target.values)

print("accuracy: ","%12.2f%%" % (score*100.0))

gs.best_params_   

10000/10000 [==============================] - 0s 26us/step
accuracy:        100.00%


{'layer_one_nodes': 20,
 'layer_two_nodes': 20,
 'num_h_layers': 1,
 'optimizer': 'adam'}

In [27]:
vat_data_inputs

,Price_net,Price_gross,Ship_from_AT,Ship_to_AT,Ship_to_CH,Ship_to_CZ,Ship_to_DE,Ship_to_FR,Ship_to_IT,Ship_to_NL,...,Buyer_AT,Buyer_CH,Buyer_CZ,Buyer_DE,Buyer_FR,Buyer_IT,Buyer_NL,VAT_ID_AT,VAT_ID_EU,VAT_ID_non-EU
0,186.06,223.27,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,105.77,126.92,1,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
2,193.41,193.41,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0
3,1263.67,1516.40,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,734.65,881.58,1,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
5,479.77,479.77,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
6,4048.56,4858.27,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,1,0,0
7,1471.78,1766.14,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
8,1246.66,1495.99,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
9,54.97,65.96,1,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [5]:
for col in vat_data_inputs.columns:
    
    vars()['x_{}_min'.format(col)], vars()['x_{}_max'.format(col)] = vat_data_inputs[col].astype('int').min() - .5, vat_data_inputs[col].astype('int').max() + .5

In [6]:
h=0.1
mesh_inputs=[]
for col in vat_data_inputs.columns:
    for col1 in vat_data_inputs.columns:
        if(vat_data_inputs[col].dtypes != np.float64):
            print(col)
            if (col != col1):
                vars()['xx_{}'.format(col)], vars()['xx_{}'.format(col1)] = np.meshgrid(np.arange(vars()['x_{}_min'.format(col)], vars()['x_{}_max'.format(col)], h),np.arange(vars()['x_{}_min'.format(col1)], vars()['x_{}_max'.format(col1)], h))  
for col in vat_data_inputs.columns:

    mesh_inputs.append(list(vars()['xx_{}'.format(col)].ravel()))


Ship_from_AT
Ship_from_AT
Ship_from_AT
Ship_from_AT
Ship_from_AT
Ship_from_AT
Ship_from_AT
Ship_from_AT
Ship_from_AT
Ship_from_AT
Ship_from_AT
Ship_from_AT
Ship_from_AT
Ship_from_AT
Ship_from_AT
Ship_from_AT
Ship_from_AT
Ship_from_AT
Ship_from_AT
Ship_from_AT
Ship_from_AT
Ship_from_AT
Ship_from_AT
Ship_from_AT
Ship_to_AT
Ship_to_AT
Ship_to_AT
Ship_to_AT
Ship_to_AT
Ship_to_AT
Ship_to_AT
Ship_to_AT
Ship_to_AT
Ship_to_AT
Ship_to_AT
Ship_to_AT
Ship_to_AT
Ship_to_AT
Ship_to_AT
Ship_to_AT
Ship_to_AT
Ship_to_AT
Ship_to_AT
Ship_to_AT
Ship_to_AT
Ship_to_AT
Ship_to_AT
Ship_to_AT
Ship_to_CH
Ship_to_CH
Ship_to_CH
Ship_to_CH
Ship_to_CH
Ship_to_CH
Ship_to_CH
Ship_to_CH
Ship_to_CH
Ship_to_CH
Ship_to_CH
Ship_to_CH
Ship_to_CH
Ship_to_CH
Ship_to_CH
Ship_to_CH
Ship_to_CH
Ship_to_CH
Ship_to_CH
Ship_to_CH
Ship_to_CH
Ship_to_CH
Ship_to_CH
Ship_to_CH
Ship_to_CZ
Ship_to_CZ
Ship_to_CZ
Ship_to_CZ
Ship_to_CZ
Ship_to_CZ
Ship_to_CZ
Ship_to_CZ
Ship_to_CZ
Ship_to_CZ
Ship_to_CZ
Ship_to_CZ
Ship_to_CZ
Ship_to_CZ
Ship_t

In [7]:
xx0, xx2 = np.meshgrid(np.arange(x0_min, x0_max, h),np.arange(x2_min, x2_max, h))  
xx0, xx3 = np.meshgrid(np.arange(x0_min, x0_max, h),np.arange(x3_min, x3_max, h))  
xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, h),np.arange(x2_min, x2_max, h))  
xx1, xx3 = np.meshgrid(np.arange(x1_min, x1_max, h),np.arange(x3_min, x3_max, h))  
xx2, xx3 = np.meshgrid(np.arange(x2_min, x2_max, h),np.arange(x3_min, x3_max, h))  
xx0, xx1 = np.meshgrid(np.arange(x0_min, x0_max, h),np.arange(x1_min, x1_max, h))
xx0, xx1 = np.meshgrid(np.arange(x0_min, x0_max, h),np.arange(x1_min, x1_max, h))  

NameError: name 'x0_min' is not defined

In [92]:
len(mesh_inputs[1])

2316800

In [85]:
np.concatenate((mesh_inputs),axis=1).shape

AxisError: axis 1 is out of bounds for array of dimension 1

In [55]:
# Plot the training points
Z = gs.predict_proba(np.concatenate(mesh_inputs))[:, 1]

Z = Z.reshape(vars()['xx_{}'.format('Ship_to_AT')].shape)

ValueError: Error when checking input: expected dense_7_input to have shape (24,) but got array with shape (1,)

In [ ]:
import plotly.graph_objs as go
import plotly
plotly.__version__

plotly.tools.set_credentials_file(username='ahmadhajmosa', api_key='pmk9ngKbvo8LtkeynAuT')


# Create a trace
trace = go.Scatter(
    x = vat_data_inputs['Taxclass_material_1'],
    y = vat_data_inputs['Ship_to_AT'],
    mode = 'markers',
    marker=dict(
        size=20,
        color = data_target, #set color equal to a variable
        colorscale='Jet',
        showscale=False
    )
)
trace2= go.Contour(
        z=Z.ravel(),
    x=xx0.ravel(),
    y=xx1.ravel(),
    showscale=False
    )

datapost = [trace,trace2]

# Plot and embed in ipython notebook!
py.iplot(datapost, filename='basic-scatter')

In [ ]:
encoded_vat_data_inputs=pd.DataFrame(encoded_vat_data_inputs)

In [ ]:
encoded_vat_data_inputs[['Price_net','Price_gross']]=vat_data_inputs[['Price_net','Price_gross']]

In [ ]:
target_encoder=sklearn.preprocessing.LabelEncoder()
target_encoder.fit(vat_data_target.values)
encoded_vat_data_target = target_encoder.transform(vat_data_target.values)

In [ ]:
encoded_vat_data_target=pd.DataFrame(encoded_vat_data_target,columns=['Taxcode'])


In [ ]:
train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(vat_data_inputs.values, vat_data_target.values, train_size=0.80)

In [ ]:
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
rf.fit(train, labels_train)